#Paquetes necesarios

In [1]:
from ultralytics import YOLO

# Carga del modelo preentrenado
model = YOLO('yolo11n.pt')

# Entrenamiento
model.train(
    data='dataset/data.yaml',   # tu archivo YAML
    epochs=50,                    # número de épocas
    imgsz=640,                    # tamaño de imagen
    batch=16,                     # ajusta según tu GPU
    device=0                      # usa GPU si tienes CUDA
)


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\mcash\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.223  Python-3.9.23 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, io

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000020A72698640>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

Modelos preentrenados, visualizando con las utilidades de ultralytics

In [26]:
import cv2
from ultralytics import YOLO
import math
import pandas as pd

# Modelos
model_general = YOLO('yolo11n.pt')  # personas y vehículos
model_plates = YOLO(r'runs\detect\\train\weights\best.pt')  # matrículas

# Nombres de las clases del modelo general (COCO)
classNames_general = [
    "person", "bicycle", "car", "motorbike", "aeroplane", "bus",
    "train", "truck"
]
clases_interes = [0, 1, 2, 3, 5, 7]  # solo personas, bicis, coches, motos, buses y camiones
clases_matricula = [2, 3, 5, 7]  # solo objetos donde buscar matrículas

video_path = 'C0142.mp4'
vid = cv2.VideoCapture(video_path)

# Configurar salida de vídeo
width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = vid.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_vid = cv2.VideoWriter('output_tracking.mp4', fourcc, fps, (width, height))

# CSV de resultados
csv_rows = []

frame_num = 0
while True:
    ret, frame = vid.read()
    if not ret:
        break

    annotated_frame = frame.copy()
    frame_num += 1

    # 1️⃣ Tracking de personas y vehículos
    results_general = model_general.track(frame, persist=True, conf=0.4, iou=0.4, classes=clases_interes, imgsz=960)
    for r in results_general:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls = int(box.cls[0])
            cls_name = classNames_general[cls] if cls < len(classNames_general) else str(cls)
            track_id = int(box.id[0]) if box.id is not None else -1
            confidence = round(float(box.conf[0]), 2)

            # Dibujar caja de objeto
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0,255,0), 3)
            cv2.putText(annotated_frame, f"{cls_name} ID:{track_id} {confidence}", 
                        (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)

            # 2️⃣ Detección de matrícula dentro de coches, motos, buses y camiones
            plate_info = {'mx1': None, 'my1': None, 'mx2': None, 'my2': None, 'plate_text': None, 'plate_conf': None}
            if cls in clases_matricula:
                pad = 5  # ampliar ROI
                roi = frame[max(0,y1-pad):y2+pad, max(0,x1-pad):x2+pad]
                results_plate = model_plates.track(roi, persist=True, conf=0.4, iou=0.4, imgsz=640)
                for rp in results_plate:
                    for b in rp.boxes:
                        px1, py1, px2, py2 = map(int, b.xyxy[0])
                        px1 += x1; px2 += x1; py1 += y1; py2 += y1
                        plate_id = int(b.id[0]) if b.id is not None else -1
                        plate_conf = round(float(b.conf[0]),2)

                        # Dibujar caja de matrícula
                        cv2.rectangle(annotated_frame, (px1, py1), (px2, py2), (0,0,255), 2)
                        cv2.putText(annotated_frame, f"Plate ID:{plate_id}", (px1, py1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)

                        # Guardar info de matrícula para CSV
                        plate_info.update({'mx1': px1, 'my1': py1, 'mx2': px2, 'my2': py2, 'plate_conf': plate_conf})

            # Guardar fila en CSV
            csv_rows.append({
                'frame': frame_num,
                'tipo_objeto': cls_name,
                'confianza': confidence,
                'id_tracking': track_id,
                'x1': x1, 'y1': y1, 'x2': x2, 'y2': y2,
                'mx1': plate_info['mx1'],
                'my1': plate_info['my1'],
                'mx2': plate_info['mx2'],
                'my2': plate_info['my2'],
                'plate_text': plate_info['plate_text'],
                'plate_conf': plate_info['plate_conf']
            })

    # Escribir fotograma en vídeo
    out_vid.write(annotated_frame)

    # Mostrar en pantalla
    cv2.imshow("Detección y Tracking", annotated_frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC para salir
        break

vid.release()
out_vid.release()
cv2.destroyAllWindows()

# Guardar CSV
import pandas as pd
df = pd.DataFrame(csv_rows)
df.to_csv('detecciones_tracking.csv', index=False)
print("Vídeo y CSV generados con éxito.")



0: 544x960 1 person, 3 cars, 1 bus, 40.1ms
Speed: 6.8ms preprocess, 40.1ms inference, 2.6ms postprocess per image at shape (1, 3, 544, 960)

0: 544x640 1 plate', 22.4ms
Speed: 4.0ms preprocess, 22.4ms inference, 4.3ms postprocess per image at shape (1, 3, 544, 640)

0: 512x640 (no detections), 22.6ms
Speed: 3.0ms preprocess, 22.6ms inference, 1.9ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 (no detections), 18.5ms
Speed: 2.7ms preprocess, 18.5ms inference, 0.9ms postprocess per image at shape (1, 3, 512, 640)

0: 480x640 (no detections), 24.7ms
Speed: 4.3ms preprocess, 24.7ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 544x960 1 person, 3 cars, 1 bus, 62.6ms
Speed: 5.2ms preprocess, 62.6ms inference, 1.7ms postprocess per image at shape (1, 3, 544, 960)

0: 544x640 (no detections), 13.5ms
Speed: 2.8ms preprocess, 13.5ms inference, 0.6ms postprocess per image at shape (1, 3, 544, 640)

0: 512x640 (no detections), 5.6ms
Speed: 1.1ms preprocess, 

Desde cámara, detección con yolo11, modelo nano. Visualización propia con OpenCV

In [ ]:
# Carga del modelo, descarga en disco si no está presente en la carpeta
model = YOLO('yolo11n.pt') #Contenedores

# Etiqueta de las distintas clases
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]


# Captura desde la webcam
vid = cv2.VideoCapture(0)
  
while(True):      
    # fotograma a fotograma
    ret, img = vid.read()
  
    # si hay imagen válida
    if ret:  
        # Detecta en la imagen
        results = model(img, stream=True)
        
        # Para cada detección
        for r in results:
            boxes = r.boxes

            for box in boxes:
                # Contenedor
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values
                
                # Confianza
                confidence = math.ceil((box.conf[0]*100))/100
                print("Confianza --->",confidence)

                # Clase
                cls = int(box.cls[0])
                print("Clase -->", classNames[cls])

                # Convierte identificador numérico de clase a un color RGB
                escala = int((cls / len(classNames)) * 255 * 3)
                if escala >= 255*2:
                    R = 255
                    G = 255
                    B = escala - 255*2
                else:
                    if escala >= 255:
                        R = 255
                        G = escala - 255
                        B = 0
                    else:
                        R = escala
                        G = 0
                        B = 0

                # Dibuja el contenedor y clase
                cv2.rectangle(img, (x1, y1), (x2, y2), (R, G, B), 3)
                cv2.putText(img, classNames[cls] , [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, B), 2)

        # Muestra fotograma
        cv2.imshow('Vid', img)
    
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()

Seguimiento. Requiere instalar lap con pip install lap

In [ ]:
from collections import defaultdict
import numpy as np

# Carga del modelo, descarga en disco si no está presente en la carpeta
model = YOLO('yolo11n.pt') #Contenedores

# Etiqueta de las distintas clases
classNames = ["person", "bicycle", "car"]


# Captura desde la webcam
vid = cv2.VideoCapture(0)
track_history = defaultdict(lambda: [])
  
while(True):      
    # fotograma a fotograma
    ret, img = vid.read()
  
    # si hay imagen válida
    if ret:  
        # Seguimiento, con persistencia entre fotogramas
        results = model.track(img, persist=True, classes = [0,1,2])

        if 0:
            if results is not None:
                print(results[0])
                boxes = results[0].boxes.xywh.cpu()
                track_ids = results[0].boxes.id.int().cpu().tolist()
                annotated_frame = results[0].plot()
                for box, track_id in zip(boxes, track_ids):
                    x, y, w, h = box
                    track = track_history[track_id]
                    track.append((float(x), float(y)))
                    if len(track) > 30:
                        track.pop(0)
                    points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                    cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)
                cv2.imshow("YOLO11 Tracking", annotated_frame)
                if cv2.waitKey(1) & 0xFF == ord("q"):
                    break
        

        
        # Para cada detección
        for r in results:
            boxes = r.boxes

            for box in boxes:
                # Contenedor
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

                #Etiqueta de seguimiento
                if box.id is not None:
                    track_id = str(int(box.id[0].tolist()))
                else:
                    track_id = ''
                
                # Confianza
                confidence = math.ceil((box.conf[0]*100))/100
                print("Confianza --->",confidence)

                # Clase
                cls = int(box.cls[0])
                print("Clase -->", classNames[cls])

                # Convierte identificador numérico de clase a un color RGB
                escala = int((cls / len(classNames)) * 255 * 3)
                if escala >= 255*2:
                    R = 255
                    G = 255
                    B = escala - 255*2
                else:
                    if escala >= 255:
                        R = 255
                        G = escala - 255
                        B = 0
                    else:
                        R = escala
                        G = 0
                        B = 0

                # Dibuja el contenedor y clase
                cv2.rectangle(img, (x1, y1), (x2, y2), (R, G, B), 3)
                cv2.putText(img, track_id + ' ' + classNames[cls] , [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, B), 2)

        # Muestra fotograma
        cv2.imshow('Vid', img)
    
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()



Intregración con seguimiento (tracking)
!!!!!!!!!Nota: he tenido que bajar a la versión de python 3.9.5 e instalar lap con pip install lap

In [ ]:
# Carga del modelo
model = YOLO('yolo11n.pt') #Contenedores
#model = YOLO('yolov11n-seg.pt') #Máscaras
#model = YOLO('yolo11n-pose.pt')  #Pose

#Para un vídeo 
filename = "TGC23_PdH_C0056cut.mp4"
results = model.track(source=filename, show=True)  # BoT-SORT tracker (por defecto)
#results = model.track(source=filename, show=True, tracker="bytetrack.yaml")  # ByteTrack tracker

cv2.destroyAllWindows()